[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chuangtc/ECTSum/blob/main/fine-tune-GPT3.ipynb)


In [26]:
print('Hello, AIDATATOOLS')
!pip install --upgrade openai
import os
import openai
os.environ["OPENAI_API_KEY"] = ''

Hello, AIDATATOOLS
You should consider upgrading via the '/Users/jasontgi/.pyenv/versions/3.9.16/bin/python3.9 -m pip install --upgrade pip' command.


## Do text smmarization, generate training validation data for GPT3

In [17]:
import json, requests

def parse_line_break(foo): 
  return iter(foo.splitlines())


train_http_path = "https://github.com/chuangtc/ECTSum-GPT3/blob/main/data-bert/train.json?raw=true"  
req = requests.get(train_http_path)

train_json_str_array = []
train_json_str_array = list(parse_line_break(req.text))

train_json =[]
for x in train_json_str_array: 
  tmp_json = json.loads(x)
  tmp_json['prompt'] = 'Summarize the following text:' + tmp_json.pop('doc')
  tmp_json['completion'] = tmp_json.pop('summaries')
  tmp_json.pop('labels', None)

  train_json.append(tmp_json)

val_http_path = "https://github.com/chuangtc/ECTSum-GPT3/blob/main/data-bert/val.json?raw=true"  
req2 = requests.get(val_http_path)

val_json_str_array = []
val_json_str_array = list(parse_line_break(req2.text))

val_json =[]
for x in val_json_str_array: 
  tmp_json = json.loads(x)
  tmp_json['prompt'] = 'Summarize the following text:' + tmp_json.pop('doc')
  tmp_json['completion'] = tmp_json.pop('summaries')
  
  tmp_json.pop('labels', None)

  val_json.append(tmp_json)

print('Training examples:')
#print(train_json[0])
#print(train_json[1])
print('Validation examples:')
#print(val_json[0])
#print(val_json[1])

Training examples:
Validation examples:


In [18]:
# Writing to sample.json
with open("prepared_train.jsonl", "w") as outfile:
    outfile.write('\n'.join(json.dumps(i) for i in train_json))

# Writing to sample.json
with open("prepared_val.jsonl", "w") as outfile:
    outfile.write('\n'.join(json.dumps(i) for i in val_json))

In [19]:
!openai api fine_tunes.create -t "prepared_train.jsonl" -v "prepared_val.jsonl" -m ada 

Upload progress: 100%|████████████████████| 12.4M/12.4M [00:00<00:00, 8.43Git/s]
Uploaded file from prepared_train.jsonl: file-23ru9jt8NQlwmgYT42UCJd8E
Upload progress: 100%|████████████████████| 1.77M/1.77M [00:00<00:00, 1.32Git/s]
Uploaded file from prepared_val.jsonl: file-wFCHjxi0Kmdet1baPovJB0vK
Created fine-tune: ft-3Wnb4hOrXU1FuQGDRfyvNWlz
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-01 06:54:03] Created fine-tune: ft-3Wnb4hOrXU1FuQGDRfyvNWlz

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-3Wnb4hOrXU1FuQGDRfyvNWlz



### It takes time to put fine-tune job in queue and training. Let's wait and check the status

In [24]:
!openai api fine_tunes.follow -i ft-3Wnb4hOrXU1FuQGDRfyvNWlz

[2023-03-01 06:54:03] Created fine-tune: ft-3Wnb4hOrXU1FuQGDRfyvNWlz
[2023-03-01 07:04:44] Fine-tune costs $4.33
[2023-03-01 07:04:45] Fine-tune enqueued. Queue number: 8
[2023-03-01 07:13:23] Fine-tune is in the queue. Queue number: 7
[2023-03-01 07:15:48] Fine-tune is in the queue. Queue number: 6
[2023-03-01 07:24:56] Fine-tune is in the queue. Queue number: 5
[2023-03-01 07:25:00] Fine-tune is in the queue. Queue number: 4
[2023-03-01 07:25:40] Fine-tune is in the queue. Queue number: 3
[2023-03-01 07:26:53] Fine-tune is in the queue. Queue number: 2
[2023-03-01 07:39:00] Fine-tune is in the queue. Queue number: 1
[2023-03-01 07:47:47] Fine-tune is in the queue. Queue number: 0
[2023-03-01 07:49:08] Fine-tune started
[2023-03-01 07:54:38] Completed epoch 1/4
[2023-03-01 07:59:43] Completed epoch 2/4
[2023-03-01 08:04:49] Completed epoch 3/4
[2023-03-01 08:09:54] Completed epoch 4/4
[2023-03-01 08:10:20] Uploaded model: ada:ft-tpisoftware-2023-03-01-00-10-20
[2023-03-01 08:10:21] Up

In [ ]:
!openai api completions.create -m ada:ft-tpisoftware-2023-03-01-00-10-20 -p "Summarize the following text:During the first quarter, we maintained a very safe environment with an RIR of 0.64, which is in line with our full year 2020 performance.\nFinancially, we delivered record first quarter revenue of $1.9 billion, an increase of 20% compared with the first quarter of 2020, up 18% on a constant currency basis; and adjusted EBIT of $282 million, which is more than double what we reported for the same period last year and a record for any first quarter historically.\nOur performance during the quarter was driven by good volumes, broad price realization and strong manufacturing efficiencies across all our businesses, resulting in an adjusted EBIT margin for the company of 15%, with all three of our businesses posting double-digit EBIT margins for the third consecutive quarter.\nShe joins us from Nordson Corporation where she served for 8 years, most recently as their General Counsel.\nWe began our sustainability journey nearly 2 decades ago.\nWe are proud of our progress and our accomplishments over the past decade across all of our 2020 sustainability goals, particularly our progress on climate action, where we have reduced absolute greenhouse gas emissions from our operations by 60% since our peak year despite adding several material acquisitions along the way.\nIn our recently published 2030 goals, we are committed to further reduce these emissions by another 50%.\nThis will result in 2030 absolute greenhouse gas emissions being 75% below our peak.\nAt the same time, we are also committed to a 30% reduction in our scope 3 emissions as we focus on making a positive impact throughout our supply chain.\nStrong top line growth, 400 basis points of gross margin expansion and continued operating expense discipline drove record first quarter adjusted EBIT, along with an adjusted EBIT margin of 15%.\nFor the first quarter, we reported consolidated net sales of $1.9 billion, up 20% over 2020 with double-digit revenue growth in all 3 segments, reflecting the robust US residential housing market and the continued strengthening of commercial and industrial markets.\nAdjusted EBIT for the first quarter of 2021 reached $282 million, up $166 million compared to the prior year and was highlighted by all 3 segments continuing to deliver double-digit EBIT margins.\nAdjusted earnings for the first quarter were $183 million or $1.73 per diluted share compared to $67 million or $0.62 per diluted share in Q1 2020.\nDepreciation and amortization expense for the quarter was $119 million, up slightly compared to the prior year.\nOur capital additions for the first quarter were $60 million, up $6 million as compared to Q1 2020.\nSlide 6 reconciles our first quarter adjusted EBIT of $282 million to our reported EBIT of $301 million.\nDuring the quarter, we recognized $20 million of gains on the sale of certain precious metals.\nIn addition, we recorded $1 million of restructuring costs associated with the Insulation network optimization actions that we initiated in the fourth quarter of 2020.\nAdjusted EBIT of $282 million was a new first quarter record for the company and increased $166 million over the prior year.\nRoofing and Insulation more than doubled their EBIT and Composites grew by 80%.\nSales for the quarter were $700 million, a 16% increase over first quarter 2020.\nWe delivered margins of 12% and EBIT of $82 million more than double the $39 million of EBIT in the first quarter of last year.\nSales for the first quarter were $559 million, up 13% compared to the prior year.\nFor the quarter, Composites delivered $79 million of EBIT and EBIT margin of 14%.\nSales in the first quarter were $711 million, up 28% compared to the prior year.\nThe US asphalt shingle market grew 26% for the quarter as compared to the prior year with our US shingle volumes, slightly outperforming the market.\nSimilar to the other 2 businesses, strong manufacturing performance was a fundamental element of the Roofing business results.\nFor the quarter, EBIT was $156 million, up $92 million from the prior year, achieving 22% EBIT margins.\nFree cash flow for the first quarter of 2021 at $120 million, up $264 million compared to the first quarter of 2020.\nDuring the first quarter of 2021, we repurchased 1.6 million shares of our common stock and returned $197 million of cash to shareholders through stock repurchases and dividends.\nAt quarter end, the company had liquidity of approximately $1.7 billion, consisting of $605 million of cash and nearly $1.1 billion of combined availability on our bank debt facilities.\nWe remain focused on consistently generating strong free cash flow, returning at least 50% to investors over time and maintaining an investment-grade balance sheet.\nGeneral corporate expenses are expected to range between $135 million and $145 million.\nCapital additions are expected to be approximately $460 million, which is below expected depreciation and amortization of approximately $480 million.\nInterest expense is estimated to be between $120 million and $130 million.\nAnd we expect our 2021 effective tax rate to be 26% to 28% of adjusted pre-tax earnings and our cash tax rate to be 18% to 20% of adjusted pre-tax earnings.\nGiven the decline in North American residential Fiberglas Insulation shipments last year during the second quarter, we expect those sea shipments to grow about 25%, with pricing continuing to improve from realization of our April increase.\nGiven our outlook for inflation, we have also recently announced an 8% price increase effective June 28.\nAdditionally, we anticipate benefits of approximately $30 million from better fixed cost absorption on higher production volumes.\nMoving on to Composites; we expect our volume growth to continue at a strong pace, up approximately 30% versus the prior year.\nMargins should benefit from the reversal of roughly $30 million of curtailment cost we saw in the second quarter of 2020.\nAnd in Roofing, we expect the market to be up between 15% and 20%, with our volumes up mid to high single digits.\nRoofing pricing is expected to improve with the announced increase of 5% to 7% that was effective at the beginning of this month.\nGiven this, we have recently announced an additional price increase of 4% to 6% effective in mid-June.\nOverall, we expect EBIT margins to increase sequentially from Q1, approaching mid-20%.\nReturning at least 50% of free cash flow to shareholders over time through dividends and share repurchases and maintaining an investment-grade balance sheet."

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Completion.create(
  model="text-davinci-003",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)

In [28]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Completion.create(
  model="ada:ft-tpisoftware-2023-03-01-00-10-20",
  prompt="Summarize the following text:During earining call, the price drops 10%.",
  max_tokens=256,
  temperature=0
)

<OpenAIObject text_completion id=cmpl-6p4x0eQuRB83JEWs19ssaSS5g7TKC at 0x1011ea4a0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\nWe have been in the business of providing our customers with the best quality products and services for over 40 years.\nWe have been in the business of providing our customers with the best quality products and services for over 40 years.\nWe have been in the business of providing our customers with the best quality products and services for over 40 years.\nWe have been in the business of providing our customers with the best quality products and services for over 40 years.\nWe have been in the business of providing our customers with the best quality products and services for over 40 years.\nWe have been in the business of providing our customers with the best quality products and services for over 40 years.\nWe have been in the business of providing our customers with the best quali